In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-jax/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [10]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife-torch/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 48 commands: 
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_multi_prompt_clip/cell_prompt_new_cs_seed=0_i1=0"  --dtype="float32" --device="cuda:0" --img_size=64 --n_layers=2 --d_state=16 --d_embd=32 --locality=1 --kernel_size=3 --nonlin="GELU" --init_state="point" --padding_mode="zeros" --dt=0.01 --p_drop=0.0 --rollout_steps=512 --bptt_steps=16 --pool_size=1024 --target_img_path=None --prompt="many molecules and chemicals randomly floating around;chemicals coming together to form a cell;a single cell moving around;a cell reproducing to create more cells;many cells moving around;a bustling diverse ecosystem of cells floating around;many cells coming together to form a bigger cell;bigger cells chasing smaller cells in predator-prey like dynamics" --n_augs=1 --augs="crop+pers" --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_softmax=9.999999999999999e-06  --coef_temporal=0.0 --bs=8 --lr=0.0003 --

In [11]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43


In [12]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [13]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife-torch/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43/job_00003.status
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_multi_prompt_clip/cell_prompt_new_cs_seed=0_i1=3"  --dtype="float32" --device="cuda:0" --img_size=64 --n_layers=2 --d_state=16 --d_embd=32 --locality=1 --kernel_size=3 --nonlin="GELU" --init_state="point" --padding_mode="zeros" --dt=0.01 --p_drop=0.0 --rollout_steps=512 --bptt_steps=16 --pool_size=1024 --target_img_path=None --prompt="many molecules and chemicals randomly floating around;chemicals coming together to form a cell;a single cell moving around;a cell reproducing to create more cells;many cells moving around;a bustling diverse ecosystem of cells floating around;many cells coming together to form a bigger cell;bigger cells chasing smaller cells in predator-prey like dynamics" --n_augs=1 --

In [14]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_20_43/gpu_7.sh


# Track Experiment

In [20]:
a = []
for job_id in range(len(commands)):
    try:
        with open(f'{log_dir}/job_{job_id:05d}.status', 'r') as f:
            a.append(int(f.read().strip()))
    except Exception as e:
        print(job_id, e)
        a.append(-1)
a = np.array(a)

In [21]:
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
np.array(a).mean()

0.0

In [23]:
len(commands)

36

In [9]:
for job_id in range(len(commands)):
    print(job_id)
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        print(e)

0
0%|          | 684/160000 [00:24<1:28:29, 30.01it/s, loss=-0.27]
1
0%|          | 611/160000 [00:25<1:45:44, 25.12it/s, loss=-0.268]
2
0%|          | 669/160000 [00:24<1:29:14, 29.75it/s, loss=-0.27]
3
0%|          | 601/160000 [00:25<1:40:53, 26.33it/s, loss=-0.268]
4
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00004.out'
5
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00005.out'
6
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00006.out'
7
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00007.out'
8
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00008.out'
9
[Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_28_22_06_37/job_00009.out'
10
[Errno 2] No such file or

Terminated
Terminated
Terminated
Terminated
